In [2]:
import sys, os, io, json, numpy as np, random
#from sklearn.metrics import roc_auc_score
import xgboost as xgb
import tensorflow as tf
#from split import split_graph, split_interface, make_split
import graphviz
from sklearn import metrics

In [35]:
def take_along_axis(arr, indices):
    # arr (n x m)
    m = tf.shape(features)[1]
    r = tf.range(m)
    taken_arr = tf.gather(tf.reshape(features, (-1,)), indices * m + r, axis=0)
    return taken_arr

graph = tf.Graph()
with graph.as_default():
    features = tf.placeholder(dtype=tf.float64)
    ax = tf.transpose(tf.nn.top_k(-tf.transpose(features), k=tf.shape(features)[-2]).indices)
    #sorted_thresholds = tf.gather(features, ax, axis=0)  # N x M x M 
    sorted_thresholds = take_along_axis(features, ax)

In [36]:
f_example = np.array([[1, 2, 3, 4, 5],
                     [3, 2, 4.5, -10, 5]], dtype=np.float32).T

In [37]:
with tf.Session(graph=graph) as s:
    ax_v, sf_v = s.run([ax, sorted_thresholds], {features: f_example})
print(ax_v, ax_v.shape)
print()
print(sf_v, sf_v.shape)

[[0 3]
 [1 1]
 [2 0]
 [3 2]
 [4 4]] (5, 2)

[[  1.  -10. ]
 [  2.    2. ]
 [  3.    3. ]
 [  4.    4.5]
 [  5.    5. ]] (5, 2)


In [67]:
ax1 = np.array([[1, 2, 3, 4, 5],
                [3, 2, 4, 1, 5],
                [5,4,3,2,1],
                [4,5,3,1,2]
               ], dtype=np.int32).T-1

In [68]:
ax1

array([[0, 2, 4, 3],
       [1, 1, 3, 4],
       [2, 3, 2, 2],
       [3, 0, 1, 0],
       [4, 4, 0, 1]], dtype=int32)

In [69]:
cond = ax1[:,1] < 2.5
cond*1

array([1, 1, 0, 1, 0])

In [70]:
cond[ax1]*1

array([[1, 0, 0, 1],
       [1, 1, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 1, 1]])

In [71]:
np.arange(5)[cond]

array([0, 1, 3])

In [79]:
reindex = np.cumsum(cond)-1
#reindex[ax1].ravel()[cond[ax1].ravel()]
reindex[ax1].T[cond[ax1].T].reshape((ax1.shape[1], -1)).T

array([[0, 1, 2, 2],
       [1, 2, 1, 0],
       [2, 0, 0, 1]])

In [82]:
def new_ax(ax, cond):
    reindex = np.cumsum(cond)-1
    axT = ax.T
    return reindex[axT][cond[axT]].reshape((ax1.shape[1], -1)).T

In [83]:
new_ax(ax1, cond)

array([[0, 1, 2, 2],
       [1, 2, 1, 0],
       [2, 0, 0, 1]])

# Cumsum vs nn.conv2d

In [124]:
example2 = np.array([[1,2,4,8,16,32,64],
                    [16,8,4,2,1,0,-1],
                    [1,1,1,1,1, 1, 1]
                    ]).T
def mycumsum0(inputarr):
    s = tf.shape(inputarr)
    z0 = tf.zeros((s[0]//2, s[1]), dtype=inputarr.dtype)
    z1 = tf.concat((z0,inputarr), axis=0)
    s1 = tf.shape(z1)
    reshaped = tf.reshape(z1, (1, s1[0], s1[1], 1))
    filter = tf.ones((s[0], 1, 1, 1), dtype=reshaped.dtype)
    cum = tf.nn.conv2d(reshaped, filter, strides=(1,1,1,1), padding="SAME")
    cum = tf.reshape(cum, (s1[0], s1[1]))[:s[0],:]
    return cum

graph2 = tf.Graph()
with graph2.as_default():
    inputarr = tf.placeholder(dtype=tf.float64)
    cum = mycumsum0(inputarr)

with tf.Session(graph=graph2) as s:
    cum_v, = s.run([cum], {inputarr: example2})
cum_v, cum_v.shape

(array([[  1.,  16.,   1.],
        [  3.,  24.,   2.],
        [  7.,  28.,   3.],
        [ 15.,  30.,   4.],
        [ 31.,  31.,   5.],
        [ 63.,  31.,   6.],
        [127.,  30.,   7.]]), (7, 3))